In [ ]:
# =========================
# Static Factor Model – Tuning-Pipeline (ET/EN/LGBM-Logik)
# =========================
# Master-Schalter:
#   USE_DYNAMIC_FI_PIPELINE = False -> Full FE (ifo + optionale Target-Blöcke, KEIN externes DR!)
#   USE_DYNAMIC_FI_PIPELINE = True  -> Dynamic FI (rolling Feature Importance Top-N)
# Modell: src.models.SFM.ForecastModel

import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
from itertools import product

# --- 1) Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(6):
        if (cur / "src").exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ["PROJECT_ROOT"] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)

# --- 2) Imports aus dem Repo ---
from src.config import (
    GlobalConfig,
    DEFAULT_CORR_SPEC,    # expanding
    EWMA_CORR_SPEC,       # ewma
    outputs_for_model,
)
from src.tuning import run_stageA, run_stageB
from src.io_timesplits import (
    load_target,
    load_ifo_features,
    load_full_lagged_features,
    load_rolling_importance,
)
from src.models.SFM import ForecastModel  # <— SFM-Wrapper

# --- 3) Master-Schalter & Meta ---
USE_DYNAMIC_FI_PIPELINE = False  # False => Full FE; True => Dynamic FI
SEED   = 42

if USE_DYNAMIC_FI_PIPELINE:
    MODEL_NAME = "sfm_dynamic_fi"
else:
    MODEL_NAME = "sfm"

outputs_for_model(MODEL_NAME)
print(f"Modell {MODEL_NAME} wird getunt.")

# --- 4) Daten laden ---
y = load_target()             # ΔIP (DatetimeIndex)
X_ifo = load_ifo_features()   # ifo features (Full FE)

FI_BASE_DIR = PROJECT_ROOT / "outputs" / "feature_importance" / "outputs_no_missing"
if USE_DYNAMIC_FI_PIPELINE:
    try:
        X_full_lagged = load_full_lagged_features(base_dir=FI_BASE_DIR)
        rolling_imp   = load_rolling_importance(base_dir=FI_BASE_DIR)
        idx_fi = y.index.intersection(X_full_lagged.index).intersection(rolling_imp.index)
        y_fi          = y.loc[idx_fi]
        X_full_lagged = X_full_lagged.loc[idx_fi]
        rolling_imp   = rolling_imp.loc[idx_fi]
        print("Dynamic-FI Daten gefunden. Shapes:", X_full_lagged.shape, rolling_imp.shape, y_fi.shape)
    except FileNotFoundError as e:
        print("FEHLER beim Laden der Dynamic-FI Artefakte:", e)
        print("Bitte erst die FI-Jobs laufen lassen (feature_importance.ipynb).")
        raise
else:
    # Full FE: Indizes alignen
    idx_common = y.index.intersection(X_ifo.index)
    y    = y.loc[idx_common]
    X_ifo = X_ifo.loc[idx_common]
    X_full_lagged = rolling_imp = y_fi = None
    print("Full-FE Daten geladen. Shapes:", X_ifo.shape, y.shape)

# --- 5) Base-Config (wie Thesis-Policy) ---
def base_cfg_thesis() -> GlobalConfig:
    cfg = GlobalConfig(preset="thesis")  # Offizielle Splits
    cfg.policy_window   = 24
    cfg.policy_decay    = 0.95
    cfg.policy_gain_min = 0.03
    cfg.policy_cooldown = 3
    return cfg

cfg0 = base_cfg_thesis()

# --- 6) Corr-Helper ---
def make_corr_spec(kind: str) -> dict:
    if kind == "expanding":
        return dict(DEFAULT_CORR_SPEC)
    elif kind == "ewm":
        return dict(EWMA_CORR_SPEC)
    else:
        raise ValueError("kind must be 'expanding' or 'ewm'")

# --- 7) Grids -------------------------------------------------
if USE_DYNAMIC_FI_PIPELINE:
    # --------- Dynamic FI (nur Modell-HPs + n_features_to_use) ----------
    print("Erstelle HP-Grid für 'Dynamic FI'...")

    N_FEATURES_TO_USE = 20

    # SFM-HPs:
    n_factors_list = [3, 5, 8, 12]
    reg_list       = ["ols", "ridge"]
    ridge_alpha_list = [0.0, 0.1, 1.0]
    svd_solver_list  = ["auto"]  # robust & schnell; kann später erweitert werden

    weight_opts  = [{"sample_weight_decay": None},
                    {"sample_weight_decay": 0.98}]

    def build_model_grid_dynamic_fi():
        grid = []
        for k in n_factors_list:
            for reg in reg_list:
                for al in ridge_alpha_list:
                    if reg == "ols" and al != 0.0:
                        continue  # alpha nur für ridge relevant
                    for solver in svd_solver_list:
                        base = {
                            "n_features_to_use": N_FEATURES_TO_USE,
                            "n_factors": k,
                            "reg": reg,
                            "ridge_alpha": al,
                            "svd_solver": solver,
                            "fit_intercept": True,
                            "seed": SEED,
                        }
                        for w in weight_opts:
                            hp = dict(base)
                            hp.update(w)
                            grid.append(hp)
        return grid

    model_grid = build_model_grid_dynamic_fi()

else:
    # ----------------- Full FE (wie ET/EN/LGBM) ---------------------
    print("Erstelle HP-Grid für 'Full FE'...")

    # A) FE/DR-Listen
    corr_options = [
        ("expanding", make_corr_spec("expanding")),
        ("ewm",       make_corr_spec("ewm")),
    ]
    lag_candidates_list   = [(1, 2, 3, 6, 12)]
    top_k_lags_list       = [1]
    use_rm3_list          = [True]
    k1_topk_list          = [100, 300]
    redundancy_param_list = [0.90]

    # KEIN externes DR hier, weil SFM intern die PCA macht:
    dr_options_list       = [
        {"dr_method": "none"},
    ]

    # B) SFM-HPs
    n_factors_list = [3, 5, 8, 12]
    reg_list       = ["ols", "ridge"]
    ridge_alpha_list = [0.0, 0.1, 1.0]
    svd_solver_list  = ["auto"]

    # C) Target Blocks & Weighting
    target_block_options = [None, ["AR1"], ["Chronos"], ["TSFresh"]]
    weight_opts          = [{"sample_weight_decay": None}]

    def build_model_grid_full_fe():
        hp_grid = []

        fe_lists = [
            lag_candidates_list,      # lags
            top_k_lags_list,          # k_lags
            use_rm3_list,             # rm3
            k1_topk_list,             # k1
            redundancy_param_list,    # red
            dr_options_list,          # dr_opt (dict)
        ]

        for (corr_tag, corr_spec) in corr_options:
            for (lags, k_lags, rm3, k1, red, dr_opt) in product(*fe_lists):
                # gleiche Einschränkung wie ET: bei k1=100 keine DR ungleich none — hier trivial erfüllt
                if k1 == 100 and dr_opt["dr_method"] != "none":
                    continue

                base_fe = {
                    "lag_candidates": lags,
                    "top_k_lags_per_feature": k_lags,
                    "use_rm3": rm3,
                    "k1_topk": k1,
                    "redundancy_param": red,
                    **dr_opt,
                    "corr_tag": corr_tag,
                    "corr_spec": corr_spec,
                }

                for k in n_factors_list:
                    for reg in reg_list:
                        for al in ridge_alpha_list:
                            if reg == "ols" and al != 0.0:
                                continue
                            for solver in svd_solver_list:
                                base_model = {
                                    "n_factors": k,
                                    "reg": reg,
                                    "ridge_alpha": al,
                                    "svd_solver": solver,
                                    "fit_intercept": True,
                                    "seed": SEED,
                                }
                                for block_set in target_block_options:
                                    for w in weight_opts:
                                        hp = {
                                            **base_fe,
                                            **base_model,
                                            "target_block_set": block_set,
                                            **w,
                                        }
                                        hp_grid.append(hp)
        return hp_grid

    model_grid = build_model_grid_full_fe()

print("Optimierte HP-Kombinationen:", len(model_grid))
print("Erstes HP-Set:", model_grid[0] if model_grid else "Grid ist leer")

# --- 8) Stage A/B Lauf (wie ET/EN/LGBM) ----------------------------------
if model_grid:
    if USE_DYNAMIC_FI_PIPELINE:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,   # Platzhalter
            y=y_fi,
            cfg=cfg0,
            keep_top_k_final=min(5, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//4),
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,  # Platzhalter
            y=y_fi,
            cfg=cfg0,
            X_full_lagged=X_full_lagged,
            rolling_imp=rolling_imp,
        )
    else:
        shortlist = run_stageA(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            model_grid=model_grid,
            X=X_ifo,
            y=y,
            cfg=cfg0,
            keep_top_k_final=min(5, len(model_grid)),
            min_survivors_per_block=max(1, len(model_grid)//4),
        )
        run_stageB(
            model_name=MODEL_NAME,
            model_ctor=lambda hp: ForecastModel(hp),
            shortlist=shortlist,
            X=X_ifo,
            y=y,
            cfg=cfg0,
        )
else:
    print("Keine gültigen HP-Kombinationen gefunden, Stages übersprungen.")

print(f"\nDone. Check outputs/stageA|stageB/{MODEL_NAME} for results.")
